In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import torch

# Read Data

In [2]:
data = pd.read_csv("landmarks.csv",encoding = "ISO-8859-1", delimiter=",", header=None)
print(data.head())

  0           1         2         3         4         5         6     \
0  hej  frameNbr:0  0.503158  0.180337 -0.743345  0.999878  0.518581   
1  hej  frameNbr:1  0.500708  0.182518 -0.786058  0.999883  0.517591   
2  hej  frameNbr:2  0.498702  0.192451 -0.860069  0.999890  0.516010   
3  hej  frameNbr:3  0.495583  0.202635 -0.904024  0.999900  0.513235   
4  hej  frameNbr:4  0.494591  0.204605 -0.858029  0.999906  0.510880   

       7         8         9     ...      1626      1627      1628      1629  \
0  0.140605 -0.702544  0.999813  ... -0.030246  0.499487  0.846327 -0.034374   
1  0.142764 -0.741179  0.999822  ... -0.023965  0.489006  0.844206 -0.028620   
2  0.150829 -0.820234  0.999833  ...  0.000000  0.000000  0.000000  0.000000   
3  0.157958 -0.861331  0.999848  ...  0.000000  0.000000  0.000000  0.000000   
4  0.159458 -0.832953  0.999859  ... -0.019417  0.360299  0.143479 -0.025300   

       1630      1631      1632      1633      1634      1635  
0  0.518703  0.840132 

# Split data and format for RNN

In [3]:
clip_idxs = np.argwhere(data.iloc[:,1].to_numpy()=='frameNbr:0')
clip_idxs = np.append(clip_idxs,data.shape[0])
labels = data.iloc[:,0].to_numpy()
y = label_encoder.fit_transform(labels)
class_names = label_encoder.classes_
device = "cpu"

In [5]:
train_data = []
val_data = []
test_data = []
np.random.seed(0)
for i in range(len(clip_idxs)-1):
    rand = np.random.rand()
    if rand<0.6:
        X_train = torch.from_numpy(data.iloc[clip_idxs[i]:clip_idxs[i+1],2:].to_numpy()).float().to(device)
        y_train = torch.from_numpy(y[clip_idxs[i]:clip_idxs[i]+1]).long().to(device)
        train_data.append((X_train,y_train))
    elif rand<0.8:
        X_val = torch.from_numpy(data.iloc[clip_idxs[i]:clip_idxs[i+1],2:].to_numpy()).float().to(device)
        y_val = torch.from_numpy(y[clip_idxs[i]:clip_idxs[i]+1]).long().to(device)
        val_data.append((X_val,y_val))
    else:
        X_test = torch.from_numpy(data.iloc[clip_idxs[i]:clip_idxs[i+1],2:].to_numpy()).float().to(device)
        y_test = torch.from_numpy(y[clip_idxs[i]:clip_idxs[i]+1]).long().to(device)
        test_data.append((X_test,y_test))


# Create RNN

In [6]:
from MTO_SW_LSTM import MTO_SW_LSTM

window_size = 2
hidden_size = 10
num_layers = 1
n_features = 1634
stride = 1
bsize = 10
device = 'cpu'
bidir = False
nout = [100, 4]
dropout = 0.1
dropout2 = 0.2

rnn = MTO_SW_LSTM(window_size,hidden_size,num_layers,n_features,stride,bsize,device,bidir,nout,dropout,dropout2)
print(rnn)

MTO_SW_LSTM(
  (rnn): LSTM(3268, 10, batch_first=True, dropout=0.1)
  (dnn): Sequential(
    (fc0): Linear(in_features=10, out_features=100, bias=True)
    (do2): Dropout(p=0.2, inplace=False)
    (af2): Tanh()
    (lin2): Linear(in_features=100, out_features=4, bias=True)
  )
)


/home/ckjellson/anaconda3/envs/tecken/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
epochs = 100
learning_rate = 0.0001
using_gpu = False
testnbr = 1
rnn.train_model(train_data, val_data, epochs, bsize, learning_rate, using_gpu, testnbr)

Epoch 1 - Training loss: 1.3964620729287465 - Validation loss: 1.3564848899841309
Epoch 2 - Training loss: 1.3396759529908497 - Validation loss: 1.3235909938812256
Epoch 3 - Training loss: 1.3161173164844513 - Validation loss: 1.3149713277816772
Epoch 4 - Training loss: 1.306538258989652 - Validation loss: 1.2853120565414429
Epoch 5 - Training loss: 1.2851731479167938 - Validation loss: 1.2676494121551514
Epoch 6 - Training loss: 1.2626171658436458 - Validation loss: 1.2796893119812012
Epoch 7 - Training loss: 1.2722961902618408 - Validation loss: 1.2518397569656372
Epoch 8 - Training loss: 1.237175037463506 - Validation loss: 1.2337125539779663
Epoch 9 - Training loss: 1.2259008536736171 - Validation loss: 1.2268424034118652
Epoch 10 - Training loss: 1.2112764517466228 - Validation loss: 1.2152807712554932
Epoch 11 - Training loss: 1.1897220636407535 - Validation loss: 1.2154200077056885
Epoch 12 - Training loss: 1.1881271998087566 - Validation loss: 1.1882588863372803
Epoch 13 - Trai

# Get prediction probabilities

In [8]:
probs, out, y_test = rnn.test_model(test_data,using_gpu)

# Compute and print accuracy

In [9]:
preds = torch.argmax(probs,dim=1).numpy()
truth = np.array([y.numpy() for y in y_test])
from sklearn.metrics import classification_report
print(classification_report(truth,preds,target_names=class_names))

              precision    recall  f1-score   support

         hej       0.72      0.87      0.79        15
       kaffe       0.86      0.90      0.88        21
       mjölk       1.00      0.79      0.88        24
        tack       0.75      0.80      0.77        15

    accuracy                           0.84        75
   macro avg       0.83      0.84      0.83        75
weighted avg       0.86      0.84      0.84        75

